In [64]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_decomposition import PLSRegression, PLSCanonical
from sklearn.model_selection import StratifiedShuffleSplit, RepeatedStratifiedKFold

from ipywidgets import interact, interact_manual

import xlrd

# Скачаем данные результатов анализа пациентов

In [56]:
PacientData = pd.read_excel("../Data/20.06.25 список образцов Моча.xlsx", header=2)

In [57]:
Anomaly_labels = PacientData.filter(regex='_a$').columns
PacientData.drop(Anomaly_labels, axis=1, inplace=True)


PacientData.drop([f"comment{x}" for x in range(1,4)], axis=1, inplace=True)# Drop comment columns
# PacientData.drop("Leukocyte_a.1", axis=1, inplace=True) # Drop Leukocyte anomaly_detected values
PacientData = PacientData.iloc[:,:-2] # Drop Unnamed_N

In [58]:
PacientData.dropna(inplace=True)

In [61]:
PacientData

,Dataset,BAR-CODE,Gender,Age,Colour,Density,pH,Protein,Bilirubin,Glucose,...,Blood,Erythrocyte,Leukocyte_cytometry,Squamous cells,Hyaline cylinders,Bacteria,Crystals,Ferment,Small cells,Pathological cylinders
0,Chm1,10178199,m,24.0,COLORLESS,1.006,6.5,0.2,0.0,0.0,...,0.3,20.2,384.8,2.4,7.38,1430.7,0.3,0.0,0.2,1.29
1,Chm2,10152917,f,62.0,YELLOW,1.028,5.5,0.3,0.0,0.0,...,0.3,49.1,21.7,9.3,9.33,1.8,156.0,0.0,6.3,3.11
2,Chm3,10152925,f,71.0,YELLOW,1.009,5.0,0.0,0.0,0.0,...,0.0,2.5,48.0,7.5,0.25,4.6,0.0,0.0,2.9,0.12
3,Chm4,10162383,f,57.0,YELLOW,1.018,5.0,0.0,0.0,0.0,...,0.0,4.5,27.8,12.5,0.90,17214.4,0.1,0.0,2.7,0.64
4,Chm5,10163935,m,77.0,LIGHT YELLOW,1.029,6.0,0.0,0.0,4.0,...,0.0,2.4,3.6,3.3,0.00,59.8,0.0,0.0,1.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,Chm496,20204362,m,68.0,LIGHT YELLOW,1.006,6.5,0.0,0.0,0.0,...,0.0,1.5,2.5,1.0,0.00,4.5,0.0,0.0,0.9,0.00
486,Chm497,20204361,m,52.0,YELLOW,1.032,5.5,0.2,0.0,0.0,...,0.0,4.7,2.5,1.6,0.12,6.4,39.9,0.0,1.4,0.00
487,Chm498,20204360,m,82.0,LIGHT YELLOW,1.017,7.0,0.1,0.0,0.0,...,0.0,5.9,10.4,2.9,0.38,43.3,0.2,0.0,1.6,0.25
488,Chm499,20204359,m,28.0,LIGHT YELLOW,1.011,6.5,0.0,0.0,0.0,...,0.0,3.3,3.4,1.4,0.00,2.7,0.0,0.0,1.4,0.00


In [70]:
PacientData.describe()

,BAR-CODE,Age,Density,pH,Protein,Bilirubin,Glucose,Ketones,Leukocyte,Nitrite,...,Blood,Erythrocyte,Leukocyte_cytometry,Squamous cells,Hyaline cylinders,Bacteria,Crystals,Ferment,Small cells,Pathological cylinders
count,4.280000e+02,428.000000,428.000000,428.000000,428.000000,428.0,428.000000,428.000000,428.000000,428.000000,...,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000
mean,1.844626e+07,51.609813,1.015444,5.877336,0.057477,0.0,0.088785,0.029206,25.116822,0.058411,...,0.164252,14.924065,24.028738,4.767991,0.893435,987.373364,12.449766,1.942991,1.964860,0.167687
std,4.398824e+06,21.402567,0.008657,0.631548,0.117009,0.0,0.565657,0.204658,90.016836,0.333668,...,0.777890,69.234000,91.830280,8.312059,7.689550,4496.041745,131.076290,20.281234,4.221953,0.377472
min,1.015292e+07,18.000000,1.000000,5.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.020451e+07,27.000000,1.009000,5.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.300000,1.400000,1.000000,0.000000,1.800000,0.000000,0.000000,0.500000,0.000000
50%,2.028722e+07,57.500000,1.014000,5.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,4.400000,3.250000,1.900000,0.120000,8.200000,0.100000,0.000000,1.000000,0.000000
75%,2.029071e+07,69.000000,1.021000,6.000000,0.100000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,9.050000,10.475000,4.700000,0.380000,60.275000,0.200000,0.000000,1.900000,0.120000
max,3.033392e+07,89.000000,1.042000,8.500000,1.000000,0.0,4.000000,3.000000,500.000000,2.000000,...,10.000000,1260.200000,1072.400000,79.800000,156.600000,48223.300000,2500.400000,321.500000,52.800000,3.110000


In [91]:
@interact
def plot(Parameter=PacientData.columns[5:]):
    plt.figure(figsize=(15,9))
    plt.xlim(np.min(PacientData[Parameter]), np.quantile(PacientData[Parameter], 1.0))
    plt.yscale('log')
    sns.distplot(PacientData[Parameter].values, bins=200, kde=False)

interactive(children=(Dropdown(description='Parameter', options=('Density', 'pH', 'Protein', 'Bilirubin', 'Glu…